In [3]:
import pyodbc
import pandas as pd
# Set up your connection string
from dotenv import load_dotenv
import os 

load_dotenv()

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")


query = "SELECT * FROM SalesLT.customer"

try:
    # Read data into a DataFrame
    df = pd.read_sql(query, conn)
    print("Data retrieved successfully!")
except Exception as e:
    print(f"Error reading data: {e}")

conn.close()
print("Connection closed.")

### Prueba ###
data=df.head(5)
data2=data.copy()


Connected to the Azure SQL Database successfully!


C:\Users\angel\AppData\Local\Temp\ipykernel_22188\2796495651.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data retrieved successfully!
Connection closed.


In [5]:
data_dict = data.to_dict(orient='list')

In [8]:
#Transform the 'ModifiedDate' column to string
for i in range(len(data_dict['ModifiedDate'])):
    data_dict['ModifiedDate'][i] = str(data_dict['ModifiedDate'][i])
data_dict

{'CustomerID': [1, 2, 3, 4, 5],
 'NameStyle': [False, False, False, False, False],
 'Title': ['Mr.', 'Mr.', 'Ms.', 'Ms.', 'Mr.'],
 'FirstName': ['Orlando', 'Keith', 'Donna', 'Janet', 'Lucy'],
 'MiddleName': ['N.', None, 'F.', 'M.', None],
 'LastName': ['Gee', 'Harris', 'Carreras', 'Gates', 'Harrington'],
 'Suffix': [None, None, None, None, None],
 'CompanyName': ['A Bike Store',
  'Progressive Sports',
  'Advanced Bike Components',
  'Modular Cycle Systems',
  'Metropolitan Sports Supply'],
 'SalesPerson': ['adventure-works\\pamela0',
  'adventure-works\\david8',
  'adventure-works\\jillian0',
  'adventure-works\\jillian0',
  'adventure-works\\shu0'],
 'EmailAddress': ['orlando0@adventure-works.com',
  'keith0@adventure-works.com',
  'donna0@adventure-works.com',
  'janet1@adventure-works.com',
  'lucy0@adventure-works.com'],
 'Phone': ['245-555-0173',
  '170-555-0127',
  '279-555-0130',
  '710-555-0173',
  '828-555-0186'],
 'PasswordHash': ['L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w

In [ ]:
def prepare_data(df:pd.DataFrame) -> pd.DataFrame:
    df = df[['FirstName', 'ModifiedDate']].sort_values(by='ModifiedDate')
    df['FirstNameLen'] = df['FirstName'].str.len()
    df["ModifiedDate"] = pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()
    df["ModifiedDate"] = df["ModifiedDate"].dt.days

data = prepare_data(data)
data_dict = data.to_dict(orient='list')
    

In [9]:
import json

#Formateado para la API:
data_json = json.dumps({"data": [data_dict]})

suri = open("uri2.json", "r")
scoring_uri = json.load(suri)["URI"][0]
suri.close()

In [10]:
headers = {"Content-Type": "application/json"}
import requests
response = requests.post(scoring_uri, data=data_json, headers=headers)

if response.status_code == 200:
  result = json.loads(response.json())
  print('Predicciones del modelo:',result)
  data["Exited"] = result
  print('Variable objetivo real:',list(data2['ModifiedDate']))
  #Print la precisión del modelo usando el porcentaje de aciertos
  aciertos = 0
  for i in range(len(data)):
    if data["Exited"][i] == list(data2['ModifiedDate'])[i]:
      aciertos += 1
  precision = aciertos / len(data) * 100
  print(f"Precisión del modelo: {precision:.2f}%")
else:
  print(f"Error: {response.text}")

Predicciones del modelo: ['2006-07-30', '2006-07-06', '2006-07-06', '2006-07-06', '2006-06-23']
Variable objetivo real: [Timestamp('2005-08-01 00:00:00'), Timestamp('2006-08-01 00:00:00'), Timestamp('2005-09-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2006-09-01 00:00:00')]
Precisión del modelo: 0.00%


C:\Users\angel\AppData\Local\Temp\ipykernel_22188\2714254649.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Exited"] = result
